<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Overview</a></span></li></ul></li><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Load the Data</a></span></li><li><span><a href="#Create-a-DD-SIMCA-Model" data-toc-modified-id="Create-a-DD-SIMCA-Model-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Create a DD-SIMCA Model</a></span></li><li><span><a href="#Load-the-Data" data-toc-modified-id="Load-the-Data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Load the Data</a></span></li></ul></div>

In [6]:
using_colab = 'google.colab' in str(get_ipython())
if using_colab:
    !git clone https://github.com/mahynski/chemometrics.git
    !cd chemometrics; pip install -r requirements.txt
else:
    import sys
    sys.path.append('../../')

import chemometrics

import matplotlib.pyplot as plt
%matplotlib inline

import watermark
%load_ext watermark

%load_ext autoreload
%autoreload 2

import shap

ModuleNotFoundError: No module named 'chemometrics'

In [ ]:
import imblearn
import sklearn

from sklearn.model_selection import GridSearchCV

import numpy as np
import pandas as pd

Overview
--------
This notebook demonstrates the use of SHAP to explain model predictions.

Notes from SHAP documentation:

* "Because this is a small dataset with only a few features we use the entire training dataset for the background. In problems with more features we would want to pass only the median of the training dataset, or weighted k-medians."

In [ ]:
%watermark -t -m -v --iversions

# Load the Data

In [ ]:
# Let's load some data from the tests/ for this example
if using_colab:
    loc = 'https://raw.githubusercontent.com/mahynski/chemometrics/master/tests/data/simca_train.csv'
else:
    loc = '../tests/data/simca_train.csv'
df = pd.read_csv(loc)

In [ ]:
# You can see that samples are rows, columns are different features
df.head()

In [ ]:
raw_x = np.array(df.values[:,3:], dtype=float) # Extract features
raw_y = np.array(df['Class'].values, dtype=str) # Take the class as the target

# Create a DD-SIMCA Model

In [ ]:
from chemometrics.classifier.simca import DDSIMCA_Model

In [ ]:
# Here the data is spectra so we will not scale the X data
dds = DDSIMCA_Model(n_components=7, alpha=0.05, gamma=0.01, scale_x=False)
_ = dds.fit(raw_x, raw_y)
_ = dds.visualize(raw_x, raw_y)

In [ ]:
# How to use shap to explain binary predictions?
# What functions / links do we need to provide for models?
# Need to have this work for general pipelines
# Set up an explainer dashboard

In [ ]:
# IRIS data set example for classification

In [ ]:
# EllipticManifold 

In [ ]:
# Even if SIMCA, etc. models make multiple assignments, it is useulf to inspet EACH individual model
# to understand why each prediciton is made, since they are independent

In [ ]:
# For discriminant methods like PLS-DA it is a little more challenging to explain

In [3]:
?shap.TreeExplainer

In [ ]:
explainer = shap.KernelExplainer(
    data = 
    model = 
    link = 
)